### import

In [1]:
import random 
import pandas as pd 
import numpy as np
import os 
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import albumentations as A 
import albumentations
import albumentations.pytorch
from albumentations.pytorch.transforms import ToTensorV2 
import torchvision.models as models
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
train_df = pd.read_csv('C:/Users/moond/open/Data/train2.csv',encoding='utf-8')
train_df.head()

,id,img_path,overview,cat1,cat2,cat3
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식


In [3]:
print(len(train_df["cat3"].unique())) # type = numpy.array

128


In [4]:
label_list = train_df["cat3"].unique().tolist()

In [5]:
len(label_list)  # type = list

128

#### 1. cat3 종류별 폴더 만들기

In [6]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
 
for i in range(len(label_list)):  # cat3 개수 만큼 
    createFolder(f'./train/{label_list[i]}')  # 폴더를 생성 

#### 2. 이미지 파일을 cat3에 맞게 각 폴더로 옮기기

In [9]:
train_folder = os.listdir('./train/')
len(train_folder) # 폴더 개수 128개 생성, 이미지 16986+ 폴더 128=17114

17114

In [7]:
import shutil

In [10]:
for i in range(len(train_folder)-128):  
    
    if train_folder[i]  == "jpg":  
        label = train_df.loc[train_df["img_path"] == f"{train_folder[i]}"]["cat3"][i].item()  
        file_source = f'./train/{train_folder[i]}'  
        file_destination = f'./train/{label}/' 
        shutil.move(file_source, file_destination)  

#### 3. 이미지 전처리 (평균과 표준편차로 이미지 normalization)

In [11]:
def get_mean_std(train):
    transform = transforms.Compose([
        transforms.Resize((,)),
        transforms.ToTensor()
    ])
    
    dataset = datasets.ImageFolder(os.path.join(f'./train'), transform)
    print("데이터 정보", dataset)

    meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x,_ in dataset]
    stdRGB = [np.std(x.numpy(), axis=(1,2)) for x,_ in dataset]

    meanR = np.mean([m[0] for m in meanRGB])
    meanG = np.mean([m[1] for m in meanRGB])
    meanB = np.mean([m[2] for m in meanRGB])

    stdR = np.mean([s[0] for s in stdRGB])
    stdG = np.mean([s[1] for s in stdRGB])
    stdB = np.mean([s[2] for s in stdRGB])
    print("평균",meanR, meanG, meanB)
    print("표준편차",stdR, stdG, stdB)

SyntaxError: invalid syntax (<ipython-input-11-51a3fa626541>, line 3)

#### test data normalization

In [ ]:
def get_mean_std(test):
    test_transform = transforms.Compose([  
    transforms.Resize((, )),
    transforms.ToTensor(), 
    transforms.Normalize([], [])  
])
    dataset = datasets.ImageFolder(os.path.join(f'./test'), transform)
    print("데이터 정보", dataset)

    meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x,_ in dataset]
    stdRGB = [np.std(x.numpy(), axis=(1,2)) for x,_ in dataset]

    meanR = np.mean([m[0] for m in meanRGB])
    meanG = np.mean([m[1] for m in meanRGB])
    meanB = np.mean([m[2] for m in meanRGB])

    stdR = np.mean([s[0] for s in stdRGB])
    stdG = np.mean([s[1] for s in stdRGB])
    stdB = np.mean([s[2] for s in stdRGB])
    print("평균",meanR, meanG, meanB)
    print("표준편차",stdR, stdG, stdB)

#### 4. data augmentation 및 data loader

In [ ]:
data_dir = "./"

def image_datasets(batch_size):
    
    train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 좌우반전 
    transforms.RandomVerticalFlip(),  # 상하반전 
    transforms.Resize((, )),   
    transforms.ToTensor(), 
     transforms.Normalize([], [])
])

    train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), train_transform)

    valid_size = 0.2

    num_train = len(train_data)
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = int(np.floor(valid_size * num_train))
    train_idx, valid_idx = indices[split:], indices[:split]

    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               sampler=train_sampler,
                                               num_workers=0)


    valid_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               sampler=valid_sampler,
                                               num_workers=0)

    return train_data, train_loader, valid_loader

#### 5. cnn model